Dependencies

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import pickle

import time
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import pickle
import threading

Functions

In [2]:
def calc_time_difference(start_time):
    current_time = time.time()
    elapsed_time = current_time - start_time
    return elapsed_time

Code

In [3]:
import cv2
import time

# Modell & Encoder laden
model = load_model('/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/backend_fastapi/ai_models/BigModel_best_model.keras')
with open('/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/backend_fastapi/ai_models/Finetuned_Hot_Chocolate_Labelencoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

# Öffne die Webcam (0 ist meistens die Standard-Webcam)
cap = cv2.VideoCapture(0)
# Überprüfe, ob die Kamera erfolgreich geöffnet wurde
if not cap.isOpened():
    print("Fehler beim Öffnen der Kamera")
    exit()

# Erhalte die aktuelle FPS der Kamera
fps_of_camera = cap.get(cv2.CAP_PROP_FPS)

#Hyperparameter
num_frames = 16
frames = []
input_size = (224, 224)  # Die Größe, die dein Modell erwartet

#Testwerte
slow_counetr = 0

# Ziel FPS (Frames pro Sekunde)
target_fps = 16
frame_time = 1 / target_fps  # Zeit pro Frame in Sekunden

# Variablen zur Zeitmessung
last_time = time.time()  # Zeit des letzten Frames

# START MULTITHREADING
# Queue für das Threading, um Vorhersagen im Hintergrund zu verarbeiten
from queue import Queue
prediction_queue = Queue()

# Funktion für die Vorhersage im Hintergrund
def predict_in_background(frames_input):
    predictions = model.predict(frames_input)
    predicted_class = np.argmax(predictions, axis=1)
    class_label = encoder.inverse_transform(predicted_class)[0]
    # Das Ergebnis in die Queue legen, damit der Hauptthread es abholen kann
    prediction_queue.put(class_label)
# ENDE MULTITHREADING


#Klassenlabel für
class_label = "No Pred. yet"



while True:
    # Lese das nächste Frame
    ret, frame = cap.read()
    if not ret:
        print("Kein Frame empfangen")
        break

    #Zeit zum warten für die Gewährleistung der FPS
    time_since_last = calc_time_difference(last_time)
    wait_for_fps = frame_time - time_since_last
    if wait_for_fps > 0:
        # Warten, bis das Ziel-FPS erreicht wird
        print("Sleeping")
        time.sleep(wait_for_fps)
    else:
        print("CODE IS TO SLOW!!!!!!!!!!!!!!!!!!")
        
    
    #Testausgabe um FPS abzufragen
    test_time_passed = calc_time_difference(last_time)
    print(f'Current FPS: {round((1 / test_time_passed), 2)}')

    #Reset der Zeit nach Sleep um 16fps zu gewährleisten
    last_time = time.time()

    #Bildverarbeitung
    # Frame vorverarbeiten
    frame_resized = cv2.resize(frame, input_size)
    frame_normalized = frame_resized / 255.0  # Normalisieren

    # Hinzufügen des vorverarbeiteten Frames zur Liste
    frames.append(frame_normalized)

     # Wenn 16 Frames gesammelt wurden, führe eine Vorhersage durch
    if len(frames) == num_frames:
        # Umwandeln in ein NumPy-Array und Hinzufügen einer Batch-Dimension
        frames_input = np.array(frames)
        frames_input = np.expand_dims(frames_input, axis=0)  # (1, 16, 224, 224, 3)

        # Vorhersage im Hintergrund starten (im separaten Thread)
        prediction_thread = threading.Thread(target=predict_in_background, args=(frames_input,))
        prediction_thread.start()

        # Die Liste der Frames zurücksetzen, um neue Frames zu sammeln
        frames = []

    # Wenn Vorhersage abgeschlossen ist, das Ergebnis holen
    if not prediction_queue.empty():
        class_label = prediction_queue.get()
        print(f"Predicted: {class_label}")

    # Text auf das Frame zeichnen
    cv2.putText(frame, class_label, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 0, 0), 5)
    cv2.putText(frame, "Sequence predition: " + "most_common" , (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 0, 0), 5)
    cv2.putText(frame, "Frame predition: " + class_label , (10, 250), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 255, 0), 5)
    
    # Frame anzeigen
    cv2.imshow('Webcam for livepredction', frame)

    # Warte auf Tasteneingabe, um das Programm zu beenden
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Schließe die Kamera und das Fenster
cap.release()
cv2.destroyAllWindows()


/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/env_mac/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Sleeping
Current FPS: 14.79
CODE IS TO SLOW!!!!!!!!!!!!!!!!!!
Current FPS: 10.08
Sleeping
Current FPS: 15.77
Sleeping
Current FPS: 15.91
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.74
Sleeping


2024-12-13 08:08:19.439 Python[55591:731392] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-13 08:08:19.439 Python[55591:731392] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Current FPS: 15.74
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.76
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.7
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.97
Sleeping
Current FPS: 15.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Sleeping
Current FPS: 15.72
Predicted: take_utensils
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.85
Sleeping
Current FPS: 15.83
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.96
Sleeping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


Exception in thread Thread-5 (predict_in_background):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/env_mac/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/k8/q1ycwdlj4ll9jjq0h6hrh9hc0000gn/T/ipykernel_55591/2332556033.py", line 43, in predict_in_background
  File "/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/env_mac/lib/python3.11/site-packages/sklearn/preprocessing/_label.py", line 160, in inverse_transform
    raise ValueError("y contains previously unseen labels: %s" % str(diff))
ValueError: y

...

Current FPS: 15.72
Sleeping
Current FPS: 15.78
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.83
Sleeping
Current FPS: 15.93
Sleeping
Current FPS: 15.78
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.9
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.74
Sleeping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Current FPS: 15.74
Predicted: add_ingredient
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.71
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.71
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.04
Sleeping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Cur

Exception in thread Thread-32 (predict_in_background):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/env_mac/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/k8/q1ycwdlj4ll9jjq0h6hrh9hc0000gn/T/ipykernel_55591/2332556033.py", line 43, in predict_in_background
  File "/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/env_mac/lib/python3.11/site-packages/sklearn/preprocessing/_label.py", line 160, in inverse_transform
    raise ValueError("y contains previously unseen labels: %s" % str(diff))
ValueError: 

Current FPS: 15.72
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.76
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.71
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.74
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


Exception in thread Thread-33 (predict_in_background):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/env_mac/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/k8/q1ycwdlj4ll9jjq0h6hrh9hc0000gn/T/ipykernel_55591/2332556033.py", line 43, in predict_in_background
  File "/Users/lucacataldo/Documents/Bachelor/Complete_App/Web/env_mac/lib/python3.11/site-packages/sklearn/preprocessing/_label.py", line 160, in inverse_transform
    raise ValueError("y contains previously unseen labels: %s" % str(diff))
ValueError: 

Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.71
Sleeping
Current FPS: 15.97
Sleeping
Current FPS: 15.71
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.71
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.72
Sleeping
Current FPS: 15.73
Sleeping
Current FPS: 15.77
Sleeping
Current FPS: 15.92
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 15.74
Sleeping
Current FPS: 13.33
Sleeping
Current FPS: 14.67
CODE IS TO SLOW!!!!!!!!!!!!!!!!!!
Current FPS: 11.01
CODE IS TO SLOW!!!!!!!!!!!!!!!!!!
Current FPS: 13.15
CODE IS TO SLOW!!!!!!!!!!!!!!!!!!
Current FPS: 15.29
CODE IS TO SLOW!!!!!!!!!!!!!!!!!!
Current FPS: 11.32
CODE IS TO SLOW!!!!!!!!!!!!!!!!!!
Current FPS: 13.88
Sleeping
Current FPS: 13.9
